In [1]:
pip install elasticsearch


   ---------------------------------------- 0.0/480.2 kB ? eta -:--:--
   -- ------------------------------------- 30.7/480.2 kB 1.3 MB/s eta 0:00:01
   ----- --------------------------------- 71.7/480.2 kB 653.6 kB/s eta 0:00:01
   ----------- ---------------------------- 143.4/480.2 kB 1.1 MB/s eta 0:00:01
   ------------------- -------------------- 235.5/480.2 kB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 358.4/480.2 kB 1.6 MB/s eta 0:00:01
   ------------------------------------ --- 440.3/480.2 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 480.2/480.2 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.5 kB ? eta -:--:--
   ------------------- -------------------- 30.7/64.5 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 61.4/64.5 kB 648.1 kB/s eta 0:00:01
   ---------------------------------------- 64.5/64.5 kB 576.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch

#1 way

In [2]:
ELASTIC_PASSWORD = "biOiUqxTHDLrBMvvYT7BC9yC"
# Found in the 'Manage Deployment' page
CLOUD_ID = "ea88302d0e50445e96d4423cb86a129b:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQzZDFiOTg0NzA4MzY0MzUzYjllNjBhZWMxZjJkY2IxYyRmNGE1Njk5NGQ2ZDI0ODNiOGZhZDFmNGNmN2NkNWZlNA=="
# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# # Succclientsful rclientponse!
# client.info()

#2 way

In [5]:
# client = Elasticsearch(
#     "https://3d1b984708364353b9e60aec1f2dcb1c.us-central1.gcp.cloud.es.io:443",  # Elasticsearch endpoint
#     api_key="TlU0a0NKRUIxaEpzcERZRmt2WE46eEJNQlZPbGlSMzIyNVcxOVZUM0xIdw==",
# )
# client.info()

In [4]:
import json
with open("data_elastic.json") as file:
    for line in file:
        line = line.strip()  # Remove leading and trailing whitespaces
        if line:  # Check if the line is not empty
            try:
                document = json.loads(line)
                response = client.index(index="data_elastic_test", body=document)
            except json.JSONDecodeError as e:
                print("Error decoding JSON on line:", line)
                print("Exception:", e)

In [5]:
def create_query(search_term):
    search_term_lower = search_term.lower()
    query_body = {
    "query": {
        "bool": {
            "should": [
                {
                    "query_string": {
                        "query": "*{}*".format(search_term_lower),
                        "fields": ["name"],
                        "default_operator": "OR"
                    }
                },
                {
                    "query_string": {
                        "query": "*{}*".format(search_term_lower),
                        "fields": ["place"],
                        "default_operator": "OR"
                    }
                }
            ]
        }
        },
        "sort": [
            {
                "_score": {
                    "order": "desc"
                }
            },
            {
                "_script": {
                    "type": "number",
                    "script": {
                        "lang": "painless",
                        "source": "if (doc['name.keyword'].value == params.searchTerm) { return 1 } else { return 2 }",
                        "params": {
                            "searchTerm": search_term
                        }
                    },
                    "order": "asc"
                }
            }
        ],
        "_source": {}  
    }
    return query_body

In [6]:
def serch_in_files(search_term):
    query_body=create_query(search_term) 
    search_results = client.search(index="data_elastic_test", body=query_body)
    if search_results['hits']['total']['value'] > 0:
    # Process the hits
        for hit in search_results['hits']['hits']:
            # Access and process individual documents
            print(hit['_source'])
    else:
        print("No matching documents found.")
    

In [7]:
search_term = "Te" 
serch_in_files(search_term)

{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusattttttttem'}
{'id': 2, 'name': 'Te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'Te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'Te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'Te', 'age': 1, 'place': 'BBBBB jerusatttlem'}
{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusalem'}
{'id': 2, 'name': 'te', 'age': 1, 'place': 'BBBBB jerusattttttttlem'}
